In [1]:
import autograd.numpy as np
import pandas as pd

%load_ext autoreload
%autoreload 2

import sys
sys.path.append('..')

import inspect
import pandas as pd
import autograd.numpy as np
import matplotlib.pyplot as plt

from utils.games import WetChicken2D
from utils.models import BNN_LV, BayesianModel, SamplerModel

# Import helpers for building Weights & Biases callbacks:
from utils.training import build_wb_callback_postpred, build_wb_callback_plotfunc

In [58]:
#pip install git+https://github.com/takuseno/d4rl-atari
import gym
import time
#from ale import roms
import d4rl_atari

# env = gym.make('ALE/Breakout-v5', render_mode='human') # -v{0, 1, 2, 3, 4} for datasets with the other random seeds
env = gym.make('LunarLander-v2')

observations = []
rewards = []
actions = []

for i_episode in range(20):
    observation = env.reset()

    for t in range(100):
        env.render()
        #print(observation)
        action = env.action_space.sample() #set to no action/zero?
        observation, reward, done, info = env.step(action)
        observations.append(observation)
        rewards.append(reward)
        actions.append(action)
        print(observation, reward, done, info, action)
        if done:
            print("Episode finished after {} timesteps".format(t+1))
            break
env.close()

# interaction with its environment through dopamine-style Atari wrapper
# observation = env.reset() # observation.shape == (84, 84)
# observation, reward, terminal, info = env.step(env.action_space.sample())

# dataset will be automatically downloaded into ~/.d4rl/datasets/[GAME]/[INDEX]/[EPOCH]
# dataset = env.get_dataset()
# dataset['observations'] # observation data in (1000000, 1, 84, 84)
# dataset['actions'] # action data in (1000000,)
# dataset['rewards'] # reward data in (1000000,)
# dataset['terminals'] # terminal flags in (1000000,)

[ 2.0393371e-03  1.4143238e+00  9.7589508e-02  6.2850557e-02
 -5.5548974e-04  1.3243759e-02  0.0000000e+00  0.0000000e+00] 2.1684235575942536 False {} 1
[3.0219078e-03 1.4151384e+00 9.7587451e-02 3.6199715e-02 1.0617409e-04
 1.3234583e-02 0.0000000e+00 0.0000000e+00] 1.1624969674817578 False {} 0
[ 4.0749549e-03  1.4153451e+00  1.0643160e-01  9.1885189e-03
 -1.0057859e-03 -2.2241097e-02  0.0000000e+00  0.0000000e+00] -0.41512528976312413 False {} 3
[ 0.00498772  1.4164752   0.09308444  0.0502242  -0.00279356 -0.03575886
  0.          0.        ] -0.4862773557000139 False {} 2
[ 0.00599108  1.4170074   0.10443604  0.02364751 -0.00685574 -0.08125149
  0.          0.        ] -0.6208713545171622 False {} 3
[ 0.00690489  1.4176457   0.09593952  0.02833611 -0.01135939 -0.09008108
  0.          0.        ] -0.11028895426814528 False {} 2
[ 0.00781889  1.417684    0.09595269  0.00166071 -0.01586244 -0.09006928
  0.          0.        ] -0.04764200364022031 False {} 0
[ 0.00881424  1.4171154  

In [59]:
#     There are 8 states: the coordinates of the lander in `x` & `y`, its linear
#     velocities in `x` & `y`, its angle, its angular velocity, and two booleans
#     that represent whether each leg is in contact with the ground or not.
x_lst = []
y_lst = []
xvel_lst = []
yvel_lst = []
angle_lst = []
anglevel_lst = []
left_lst = []
right_lst = []

for i in observations:
    x_lst.append(i[0])
    y_lst.append(i[1])
    xvel_lst.append(i[2])
    yvel_lst.append(i[3])
    angle_lst.append(i[4])
    anglevel_lst.append(i[5])
    left_lst.append(i[6])
    right_lst.append(i[7])

# x_array = np.asarray(x_lst)
# y_array = np.array(y_lst)
# print(x_array)

rewards_array = np.asarray(rewards)
actions_array = np.asarray(actions)

df = pd.DataFrame(list(zip(x_lst, y_lst, xvel_lst, yvel_lst, angle_lst, anglevel_lst,
                           left_lst, right_lst, actions_array)), columns=['x', 'y', 'x velocity',
                                                                          'y velocity', 'angle', 'angular velocity', 'left_ground', 'right_ground','action'])
df.head()
# coordinates = np.concatenate((x_array, y_array))

,x,y,x velocity,y velocity,angle,angular velocity,left_ground,right_ground,action
0,0.002039,1.414324,0.097590,0.062851,-0.000555,0.013244,0.0,0.0,1
1,0.003022,1.415138,0.097587,0.036200,0.000106,0.013235,0.0,0.0,0
2,0.004075,1.415345,0.106432,0.009189,-0.001006,-0.022241,0.0,0.0,3
3,0.004988,1.416475,0.093084,0.050224,-0.002794,-0.035759,0.0,0.0,2
4,0.005991,1.417007,0.104436,0.023648,-0.006856,-0.081251,0.0,0.0,3


In [64]:
X_train_atari = df[['x', 'y', 'action']].to_numpy()
Y_train_atari = df[['x velocity', 'y velocity']].to_numpy()
X_test_atari = np.linspace(-6,6,100)

print('X :',X_train_atari.shape)
print('Y :',Y_train_atari.shape)

X : (1759, 3)
Y : (1759, 2)


In [61]:
# Define Bayesian Neural Network with Latent Variable (BNN_LV):
L = 1
N, M = X_train_atari.shape  # Input shape.
_,K = Y_train_atari.shape  # Output shape.
gamma = 1.0  # Standard deviation of noise for each latent input.
sigma = 1.0  # Standard deviation of noise on each model output.

# Newtork architecture:
architecture = {
    'input_n' : M,  # 3 inputs.
    'output_n' : K,  # 2 outputs.
    'hidden_layers' : [20,20],
    'biases' : [1,1,1],
    'activations' : ['relu', 'relu', 'linear'],
    'gamma' : [gamma]*L,
    'sigma' : [sigma]*K,
    'seed' : 207,
}

# Initialize network:
bnn_lv = BNN_LV(architecture=architecture)

# Get number of weights in network:
D = bnn_lv.D

# # Train network to get MLE estimate as starting point for sampler:
# bnn_lv.fit(X_train, Y_train, step_size=0.01, max_iteration=5000, check_point=500, regularization_coef=None)